# Grouped TCA

In [86]:
import tensortools as tt
import numpy as np
import flow
from flow.misc import wordhash
import pool
import pandas as pd
import os
from cascade import utils
from cascade import paths
from cascade import tca
from copy import deepcopy
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
mouse='OA27'
tags='naive'
run_type='training'

In [ ]:
dprime_threhsold = 2
if dprime_threhsold:
    print('yes')
dprime = []
for day1 in days:
    print(pool.calc.behavior.dprime(day1))
    dprime.append(pool.calc.behavior.dprime(day1))
# days = days[dprime > dprime_threshold]

False

In [5]:
runs = flow.metadata.RunSorter.frommeta(mice=[mouse], run_types=run_type, tags=tags)

In [9]:
runs[0:-1]

[Run(mouse='OA27', date=170116, run=3, run_type='training', tags=('hungry', 'kelly', 'naive')),
 Run(mouse='OA27', date=170118, run=3, run_type='training', tags=('hungry', 'kelly', 'naive')),
 Run(mouse='OA27', date=170119, run=1, run_type='training', tags=('hungry', 'kelly', 'naive'))]

In [9]:
# days = flow.metadata.DateSorter.frommeta(mice=[mouse], tags=tags)

# # maybe use a run sorter?

# dprime pool.calc.behavior.dprime(day1)
# pool.calc.driven.trial(day1, dcs)
run_list = []
for c, run in enumerate(runs[0:-1]):
    dfr = tca._triggerfromrun(run)
#     dfr = dfr.reset_index()[['trial_idx', 'cell_idx', 'timestamp', 'trace']]
#     dfr.rename(columns={'trace': 'trace_' + str(c)}, inplace=True)
    run_list.append(dfr)
dfr = pd.concat(run_list, axis=0)

0.0
44.0
1491.0


In [21]:
dfr.head()

trace
mouse date   run trial_idx cell_idx timestamp          
OA27  170116 3   0.0       1.0      -1.0      -0.078952
                 1.0       1.0      -1.0      -0.024965
                 2.0       1.0      -1.0       0.057309
                 3.0       1.0      -1.0      -0.248164
                 4.0       1.0      -1.0       0.101081

In [49]:
# build list of cells with their ids and then do an outer merge of the dataframes
times = len(np.unique(dfr.reset_index()['timestamp']))
test = dfr.pivot_table(
            index=['date', 'run', 'cell_idx'],
            columns=['trial_idx', 'timestamp'], values='trace')

for d in np.unique(test.reset_index()['date']):
    keep = np.where((test.reset_index()['date'] == d) == True)[0]
    df_today = test.iloc[keep, :]
    for r in np.unique(df_today.reset_index()['run']):
        keep2 = np.where((df_today.reset_index()['run'] == r) == True)[0]
        df_run = df_today.iloc[keep2, :]
        break
    break

In [54]:
# print(times)
mat = df_run.values
np.shape(mat)
# np.shape(df_run.values.reshape((np.shape(df_run)[0], times, -1)))

(264, 16816)

In [32]:
#  df_today = test.loc[(test.reset_index()['date'] == d), :]
print(np.shape(test))
print(np.shape((test.reset_index()['date'] == d)))

(754, 16816)
(754,)


In [18]:
# pd.merge(run_list[0], run_list[1], how='outer')
# test = run_list[0].reset_index()[['trial_idx', 'cell_idx', 'timestamp', 'trace_0']]
len(np.unique(test.reset_index()['cell_idx']))

446

In [51]:
test = test.pivot_table(
            index=['cell_idx'],
            columns=['trial_idx', 'timestamp'], values='trace')

In [ ]:
# np.shape(test.values)
pd.merge(run_list[0].head(), run_list[1].head(), how='outer', on='cell_idx')

In [ ]:
## using old tensor algo rather than pandas df

In [147]:
# function for getting ids, ids_bool, and ids_sorter for number of consecutive days
# count as driven if driven on any day
def _group_drive_ids(days, dcs, drive_threshold):
    """ 
    Get an array of all unique ids driven on any day for a given DaySorter.
    """
    
    good_ids = []
    for day1 in days:
    # get cell_ids
        d1_ids = flow.xday._read_crossday_ids(day1.mouse, day1.date)
        d1_ids = np.array([int(s) for s in d1_ids])

        # filter cells based on visual/trial drive across all cs

        d1_drive = []
        for dcs in drive_css:
            try:
                d1_drive.append(pool.calc.driven.trial(day1, dcs))
            except KeyError:
                print(str(day1) + ' requested ' + dcs +
                      ': no match to what was shown (probably pav only).')
        d1_drive = np.max(d1_drive, axis=0)
        d1_ids_bool = np.array(d1_drive) > drive_threshold
        d1_drive_ids = d1_ids[np.array(d1_drive) > drive_threshold]
        good_ids.extend(d1_drive_ids)

    return np.unique(good_ids)

In [148]:
mouse='OA27'
tags='naive'

# TCA params
rank=20
method=('ncp_bcd',)
replicates=3
fit_options=None

# grouping params
group_by='naive'
use_dprime=False

# tensor params
trace_type='zscore_day'
cs=''
downsample=True
start_time=-1
end_time=6
clean_artifacts=None
thresh=20
warp=False
smooth=True
smooth_win=5
verbose=True

# filtering params
exclude_tags=('disengaged', 'orientation_mapping', 'contrast', 'retinotopy', 'sated', 'learning_start')
exclude_conds=('blank', 'blank_reward', 'pavlovian')
driven=True
drive_css=('0', '135', '270')
drive_threshold=15
"""
Perform tensor component analysis (TCA) on data aligned
across a group of days. Builds one large tensor.

Algortitms from https://github.com/ahwillia/tensortools.

Parameters
-------
methods, tuple of str
    'cp_als', fits CP Decomposition using Alternating
        Least Squares (ALS).
    'ncp_bcd', fits nonnegative CP Decomposition using
        the Block Coordinate Descent (BCD) Method.
    'ncp_hals', fits nonnegtaive CP Decomposition using
        the Hierarcial Alternating Least Squares
        (HALS) Method.
    'mcp_als', fits CP Decomposition with missing data using
        Alternating Least Squares (ALS).

rank, int
    number of components you wish to fit

replicates, int
    number of initializations/iterations fitting for each rank

Returns
-------

"""

# create folder structure and save dir
if fit_options is None:
    fit_options = {'tol': 0.0001, 'max_iter': 500, 'verbose': False}
pars = {'tags': tags, 'rank': rank, 'method': method,
        'replicates': replicates, 'fit_options': fit_options,
        'trace_type': trace_type, 'cs': cs, 'downsample': downsample,
        'start_time': start_time, 'end_time': end_time,
        'clean_artifacts': clean_artifacts, 'thresh': thresh,
        'warp': warp, 'smooth': smooth, 'smooth_win': smooth_win,
        'exclude_tags': exclude_tags, 'exclude_conds': exclude_conds,
        'driven': driven, 'drive_css': drive_css,
        'drive_threshold': drive_threshold}
save_dir = paths.tca_path(mouse, 'group', pars=pars)

days = flow.metadata.DateSorter.frommeta(mice=[mouse], tags=tags)

meta_list = []
tensor_list = []
id_list = []
for c, day1 in enumerate(days, 0):

    # get cell_ids
    d1_ids = flow.xday._read_crossday_ids(day1.mouse, day1.date)
    d1_ids = np.array([int(s) for s in d1_ids])

    # filter cells based on visual/trial drive across all cs
#     d1_drive = np.max([pool.calc.driven.trial(day1, cs) for cs in drive_css], axis=0)

    # filter cells based on visual/trial drive across all cs, prevent
    # breaking when only pavs are shown
    if driven:
        good_ids = _group_drive_ids(days, dcs, drive_threshold)
        d1_ids_bool = np.isin(d1_ids, good_ids)
        d1_sorter = np.argsort(d1_ids[d1_ids_bool])
    else:
        d1_ids_bool = np.ones(np.shape(d1_ids)) > 0
        d1_sorter = np.argsort(d1_ids[d1_ids_bool])
    ids = d1_ids[d1_ids_bool][d1_sorter]

    # TODO add in additional filter for being able to check for quality of xday alignment

    # get all runs for both days
    d1_runs = day1.runs()

    # filter for only runs without certain tags
    d1_runs = [run for run in d1_runs if not any(np.isin(run.tags, exclude_tags))]

    # build tensors for all correct runs and trials after filtering
    if d1_runs:
        d1_tensor_list = []
        d1_meta = []
        for run in d1_runs:
            t2p = run.trace2p()
            # trigger all trials around stimulus onsets
            run_traces = utils.getcstraces(run, cs=cs, trace_type=trace_type,
                                     start_time=start_time, end_time=end_time,
                                     downsample=True, clean_artifacts=clean_artifacts,
                                     thresh=thresh, warp=warp, smooth=smooth,
                                     smooth_win=smooth_win)
            # filter and sort
            run_traces = run_traces[d1_ids_bool, :, :][d1_sorter, :, :]
            # get matched trial metadata/variables
            dfr = tca._trialmetafromrun(run)
            # subselect metadata if you are only running certain cs
            if cs != '':
                if cs == 'plus' or cs == 'minus' or cs == 'neutral':
                    dfr = dfr.loc[(dfr['condition'].isin([cs])), :]
                elif cs == '0' or cs == '135' or cs == '270':
                    dfr = dfr.loc[(dfr['orientation'].isin([cs])), :]
                else:
                    print('ERROR: cs called - "' + cs + '" - is not\
                          a valid option.')

            # subselect metadata to remove certain condtions
            if len(exclude_conds) > 0:
                dfr = dfr.loc[(~dfr['condition'].isin([exclude_conds])), :]

            # drop trials with nans and add to lists
            keep = np.sum(np.sum(np.isnan(run_traces), axis=0, keepdims=True),
                          axis=1, keepdims=True).flatten() == 0
            dfr = dfr.iloc[keep, :]
            d1_tensor_list.append(run_traces[:, :, keep])
            d1_meta.append(dfr)

        # concatenate matched cells across trials 3rd dim (aka, 2)
        tensor = np.concatenate(d1_tensor_list, axis=2)

        # concatenate all trial metadata in pd dataframe
        meta = pd.concat(d1_meta, axis=0)
        
    meta_list.append(meta)
    tensor_list.append(tensor)
    id_list.append(ids)

TCA parameters hashed: station


array([   4,    6,    7,    9,   10,   13,   14,   17,   18,   19,   22,
         23,   24,   25,   29,   30,   35,   39,   40,   41,   43,   44,
         46,   48,   50,   51,   53,   54,   55,   56,   58,   60,   62,
         64,   68,   71,   73,   76,   77,   79,   80,   82,   83,   84,
         87,   90,   91,   92,   93,   95,   97,   98,   99,  103,  104,
        106,  107,  108,  111,  113,  114,  116,  118,  119,  120,  121,
        122,  123,  125,  126,  127,  129,  131,  133,  134,  135,  136,
        137,  138,  139,  140,  141,  142,  144,  145,  147,  148,  149,
        150,  152,  160,  161,  169,  170,  171,  172,  173,  175,  177,
        178,  181,  182,  183,  186,  188,  189,  190,  191,  192,  194,
        196,  197,  198,  200,  201,  202,  203,  204,  205,  208,  209,
        210,  211,  213,  214,  216,  218,  220,  221,  222,  223,  224,
        225,  226,  227,  229,  230,  233,  235,  236,  237,  238,  239,
        240,  241,  243,  244,  245,  247,  248,  2

In [146]:
# get total trial number
meta = pd.concat(meta_list, axis=0)
trial_num = len(meta.reset_index()['trial_idx'])

# use ids to get union of ids. Use these for indexing and adding tensor together
id_union = np.unique(np.concatenate(id_list, axis=0))
cell_num = len(id_union)

# preallocate
trial_start = 0
trial_end = 0
group_tensor = np.zeros((cell_num, np.shape(tensor_list[0])[1], trial_num))
group_tensor[:] = np.nan
for i in range(len(tensor_list)):
    trial_end += np.shape(tensor_list[i])[2]
    print(trial_start, trial_end)
    for c, k in enumerate(id_list[i]):
        celln_all_trials = tensor_list[i][c, :, :]
#         print(np.shape(celln_all_trials))
        group_tensor[(id_union == k), :, trial_start:trial_end] = celln_all_trials
    trial_start += np.shape(tensor_list[i])[2]

0 155
155 310
310 465
465 620


In [124]:
print(np.shape(group_tensor))
for i in range(len(tensor_list)):
    print(np.shape(tensor_list[i]))

(251, 108, 620)
(139, 108, 155)
(128, 108, 155)
(108, 108, 155)
(108, 108, 155)


In [165]:
str(None)

'None'